<a href="https://colab.research.google.com/github/GuariJuan/FIA-TP5-P2/blob/main/FIA2023_TP05_P2_GUARI_JUAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalamos librerías
install.packages("e1071")#svm
install.packages("randomForest") #randomForest
install.packages("clock") #necesaria para instalar 'caret'
install.packages("caret") #para el particionado del df
install.packages("fastDummies") #para realizar el casteo (one-hot encoding)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘proxy’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘iterators’, ‘Rcpp’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
#Importamos librerías
library(e1071)
library(randomForest)
library(dplyr)
library(caret)
library(reshape2)
library(fastDummies)

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘dplyr’


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


Loading required package: lattice

Thank you for using fastDummies!

To acknowledge our work, please cite the package:

Kaplan, J. & Schlegel, B. (2023). fastDummies: Fast Creation of Dummy (Binary) Columns and Rows from Categorical Variables. Version 1.7.1. URL: https://github.com/jacobkap/fastDummies, https://jacobkap.github.io/fastDummies/.



In [3]:
df_credit_card <- read.csv("/content/Credit_card_limpio.csv")
head(df_credit_card)

,X,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,⋯,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Family_Members,label,Birthday_date,age,Employed
,<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>
1,1,5008827,M,Y,Y,0,180000,Pensioner,Higher education,Married,⋯,365243,1,0,0,0,2,1,1972-06-29,51,n
2,2,5008865,F,Y,Y,2,135000,Working,Secondary / secondary special,Married,⋯,-3173,1,0,0,0,4,0,1980-09-26,43,y
3,3,5008889,F,N,Y,0,247500,Commercial associate,Higher education,Separated,⋯,-1347,1,0,0,0,1,0,1977-04-20,46,y
4,4,5009000,M,Y,Y,0,157500,Working,Secondary / secondary special,Married,⋯,-828,1,0,0,0,2,0,1996-09-16,27,y
5,5,5009023,F,N,Y,2,216000,State servant,Higher education,Married,⋯,-3112,1,0,0,1,4,0,1981-08-09,42,y
6,6,5009053,F,N,Y,2,202500,Working,Higher education,Married,⋯,-2289,1,0,0,1,4,0,1993-10-12,30,y


In [5]:
#Solo dejaremos las columnas con las que vamos a trabajar
colnames(df_credit_card)
columnas_a_usar <- c('GENDER','Car_Owner','Propert_Owner','CHILDREN','Annual_income','Family_Members',
  'label','age','Employed','EDUCATION', 'Marital_status', 'Type_Income', 'Housing_type')

# Creamos un subconjunto con las columnas especificadas
df_credit_card <- df_credit_card[ , columnas_a_usar]

[1] "GENDER"         "Car_Owner"      "Propert_Owner"  "CHILDREN"      
 [5] "Annual_income"  "Family_Members" "label"          "age"           
 [9] "Employed"       "EDUCATION"      "Marital_status" "Type_Income"   
[13] "Housing_type"

In [7]:
#Verificamos que haya quedado la estructura que deseamos
str(df_credit_card)

#Movemos la columna de la etiqueta al final
df_credit_card <- df_credit_card %>% select(-label, everything(), label)

#Le cambio el tipo a factor para que no sea numerico
df_credit_card$label <- factor(df_credit_card$label)

'data.frame':	1496 obs. of  13 variables:
 $ GENDER        : chr  "M" "F" "F" "M" ...
 $ Car_Owner     : chr  "Y" "Y" "N" "Y" ...
 $ Propert_Owner : chr  "Y" "Y" "Y" "Y" ...
 $ CHILDREN      : int  0 2 0 0 2 2 1 0 0 0 ...
 $ Annual_income : num  180000 135000 247500 157500 216000 ...
 $ Family_Members: int  2 4 1 2 4 4 3 1 2 2 ...
 $ age           : int  51 43 46 27 42 30 48 43 43 57 ...
 $ Employed      : chr  "n" "y" "y" "y" ...
 $ EDUCATION     : chr  "Higher education" "Secondary / secondary special" "Higher education" "Secondary / secondary special" ...
 $ Marital_status: chr  "Married" "Married" "Separated" "Married" ...
 $ Type_Income   : chr  "Pensioner" "Working" "Commercial associate" "Working" ...
 $ Housing_type  : chr  "House / apartment" "House / apartment" "Rented apartment" "House / apartment" ...
 $ label         : Factor w/ 2 levels "0","1": 2 1 1 1 1 1 1 1 1 1 ...


Para SVM tenemos que castear algunas columnas con variables categórícas, es decir, convertims cada categoría en una columna con valores 0 o 1 dependiendo si se cumple o no la categoría.

In [8]:
columnas_a_castear <- c('EDUCATION', 'Marital_status', 'Type_Income', 'Housing_type')

# Utilizamos fastDummies para convertir las columnas en filas
df_credit_cardSVM <- fastDummies::dummy_cols(df_credit_card, select_columns = columnas_a_castear)
str(df_credit_cardSVM)

'data.frame':	1496 obs. of  33 variables:
 $ GENDER                                 : chr  "M" "F" "F" "M" ...
 $ Car_Owner                              : chr  "Y" "Y" "N" "Y" ...
 $ Propert_Owner                          : chr  "Y" "Y" "Y" "Y" ...
 $ CHILDREN                               : int  0 2 0 0 2 2 1 0 0 0 ...
 $ Annual_income                          : num  180000 135000 247500 157500 216000 ...
 $ Family_Members                         : int  2 4 1 2 4 4 3 1 2 2 ...
 $ age                                    : int  51 43 46 27 42 30 48 43 43 57 ...
 $ Employed                               : chr  "n" "y" "y" "y" ...
 $ EDUCATION                              : chr  "Higher education" "Secondary / secondary special" "Higher education" "Secondary / secondary special" ...
 $ Marital_status                         : chr  "Married" "Married" "Separated" "Married" ...
 $ Type_Income                            : chr  "Pensioner" "Working" "Commercial associate" "Working" ...
 $ Hous

In [9]:
# Eliminamos las columnas en df_credit_cardSVM
df_credit_cardSVM <- df_credit_cardSVM[, !(names(df_credit_cardSVM) %in% columnas_a_castear)]

#Movemos la columna de la etiqueta al final
df_credit_cardSVM <- df_credit_cardSVM %>% select(-label, everything(), label)

table(df_credit_cardSVM$label)


   0    1 
1336  160 

# Solucionamos el desbalanceo

Opción 1, elegir 160 filas aleatorias con label = 0

In [24]:
# Creamos un subconjunto de 160 filas con label = 0 al azar
  # Creamos un df con las filas con etiqueta 0
df_label_0 <- df_credit_card[df_credit_card$label == 0, ]
sampled_df_label_0 <- df_label_0[sample(nrow(df_label_0), 160), ]

# Creamos un subconjunto con las filas con label = 1
df_label_1 <- df_credit_card[df_credit_card$label == 1, ]

# Combinamos los subconjuntos en un DataFrame balanceado
df_balanceado160 <- rbind(sampled_df_label_0, df_label_1)

In [25]:
table(df_balanceado160$label)


  0   1 
160 160 

Opción 2, duplicamos las flas con label = 1

In [26]:
# Creamos un subconjunto con las filas con label = 1
df_label_1 <- df_credit_card[df_credit_card$label == 1, ]

# Duplicamos las filas con label = 1
duplicados_label_1 <- rbind(df_label_1, df_label_1)

# Creamos un subconjunto de 320 filas con label = 0 al azar
df_label_0 <- df_credit_card[df_credit_card$label == 0, ]
sampled_df_label_0 <- df_label_0[sample(nrow(df_label_0), 320), ]

# Combinamos los DataFrames de 320 filas obtenidos
df_balanceado320 <- rbind(duplicados_label_1, sampled_df_label_0)

In [27]:
str(df_balanceado320)

'data.frame':	640 obs. of  13 variables:
 $ GENDER        : chr  "M" "F" "F" "F" ...
 $ Car_Owner     : chr  "Y" "Y" "Y" "Y" ...
 $ Propert_Owner : chr  "Y" "N" "N" "N" ...
 $ CHILDREN      : int  0 0 0 0 0 1 1 1 0 0 ...
 $ Annual_income : num  180000 315000 315000 315000 180000 450000 450000 450000 90000 90000 ...
 $ Family_Members: int  2 2 2 2 2 3 3 1 2 2 ...
 $ age           : int  51 37 37 37 60 49 49 49 51 51 ...
 $ Employed      : chr  "n" "y" "y" "y" ...
 $ EDUCATION     : chr  "Higher education" "Higher education" "Higher education" "Higher education" ...
 $ Marital_status: chr  "Married" "Married" "Married" "Married" ...
 $ Type_Income   : chr  "Pensioner" "Commercial associate" "Commercial associate" "Commercial associate" ...
 $ Housing_type  : chr  "House / apartment" "House / apartment" "House / apartment" "House / apartment" ...
 $ label         : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...


In [28]:
table(df_balanceado320$label)


  0   1 
320 320 

Opción 3, trabajamos con el dataset desbalanceado

# Comenzamos con SVM
Creamos los df balanceados con el df resultante después del casteo

Opción 1

In [29]:
# Creamos un subconjunto de 160 filas con label = 0 al azar
df_label_0 <- df_credit_cardSVM[df_credit_cardSVM$label == 0, ]
sampled_df_label_0 <- df_label_0[sample(nrow(df_label_0), 160), ]

# Creamos un subconjunto con las filas con label = 1
df_label_1 <- df_credit_cardSVM[df_credit_cardSVM$label == 1, ]

# Combinamos los subconjuntos en un DataFrame balanceado
df_balanceado160SVM <- rbind(sampled_df_label_0, df_label_1)

In [35]:
#seteamos la semilla para que el experimento sea reproducible
set.seed(2023)
#particionamos el df en 70% para entrenamiento y 30% para prueba
t.ids <- createDataPartition(df_balanceado160SVM$label, p =0.7, list=F)
#entrenamiento
modelo <- svm(label ~ ., data = df_balanceado160SVM[t.ids, ])
#prueba
prediccion <- predict(modelo, df_balanceado160SVM[-t.ids, ])
#creamos una matriz de confusión
matriz_confusion <- table(df_balanceado160SVM$label[-t.ids], prediccion, dnn = c('Real', 'Predicho'))

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘X.Housing_type_Co.op.apartment.’ constant. Cannot scale data.”


In [36]:
matriz_confusion

    Predicho
Real  0  1
   0 39  9
   1 22 26

In [32]:
#Error calculado como (total de filas mal clasificadas/total de filas)
sum(prediccion != df_balanceado160$label[-t.ids])/length(df_balanceado160$label[-t.ids])

[1] 0.3229167

In [33]:
#Error calculado usando la matriz de confusión (equivalente a la anterior)
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.3229167

In [34]:
# Calculamos las métricas de precisión y sensibilidad
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])
print(precision)
print(sensibilidad)

[1] 0.6770833
[1] 0.5416667


Opción 2

In [37]:
# Creamos un subconjunto con las filas con label = 1
df_label_1 <- df_credit_cardSVM[df_credit_cardSVM$label == 1, ]

# Duplicamos las filas con label = 1
duplicados_label_1 <- rbind(df_label_1, df_label_1)

# Creamos un subconjunto de 1320 filas con label = 0 al azar
df_label_0 <- df_credit_cardSVM[df_credit_cardSVM$label == 0, ]
sampled_df_label_0 <- df_label_0[sample(nrow(df_label_0), 320), ]

# Combinamos los DataFrames de 320 filas obtenidos
df_balanceado320SVM <- rbind(duplicados_label_1, sampled_df_label_0)

In [40]:
set.seed(2023)
t.ids <- createDataPartition(df_balanceado320SVM$label, p =0.7, list=F)
modelo <- svm(label ~ ., data = df_balanceado320SVM[t.ids, ])
prediccion <- predict(modelo, df_balanceado320SVM[-t.ids, ])
matriz_confusion <- table(df_balanceado320SVM$label[-t.ids], prediccion, dnn = c('Real', 'Predicho'))

In [41]:
print(matriz_confusion)

    Predicho
Real  0  1
   0 59 37
   1 43 53


In [42]:
sum(prediccion != df_balanceado320$label[-t.ids])/length(df_balanceado320$label[-t.ids])

[1] 0.4166667

In [43]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.4166667

In [44]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.5833333
[1] 0.5520833


Opción 3

In [45]:
set.seed(2023)
t.ids <- createDataPartition(df_credit_cardSVM$label, p =0.7, list=F)
modelo <- svm(label ~ ., data = df_credit_cardSVM[t.ids, ])
prediccion <- predict(modelo, df_credit_cardSVM[-t.ids, ])
matriz_confusion <- table(df_credit_cardSVM$label[-t.ids], prediccion, dnn = c('Real', 'Predicho'))

In [46]:
print(matriz_confusion)

    Predicho
Real   0   1
   0 400   0
   1  48   0


In [47]:
sum(prediccion != df_credit_cardSVM$label[-t.ids])/length(df_credit_cardSVM$label[-t.ids])

[1] 0.1071429

In [48]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.1071429

In [49]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.8928571
[1] 0


# Random Forest
Trabajamos con las columnas de variables categóricas sin castear

In [50]:
set.seed(2023)
t.ids <- createDataPartition(df_credit_card$label, p =0.7, list=F)
modelo_rf <- randomForest(label ~ ., data = df_credit_card[t.ids, ])
prediccion_rf <- predict(modelo_rf, newdata = df_credit_card[-t.ids, ])
matriz_confusion <- table(prediccion_rf, df_credit_card$label[-t.ids], dnn = c('Real' ,'Predicho'))

In [51]:
matriz_confusion

    Predicho
Real   0   1
   0 394  40
   1   6   8

In [52]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.1026786

In [53]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.8973214
[1] 0.5714286


Otra opción para reducir la cantidad de filas y tratar de balancear el df sería eliminar outliers, en este caso los que estén por encima del 3er cuartil

In [54]:
summary(df_credit_card$Annual_income)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  33750  121050  166050  190775  225000 1575000 

In [55]:
df_sin_outliers <- subset(df_credit_card, Annual_income <= 225000)

In [56]:
str(df_sin_outliers)

'data.frame':	1164 obs. of  13 variables:
 $ GENDER        : chr  "M" "F" "M" "F" ...
 $ Car_Owner     : chr  "Y" "Y" "Y" "N" ...
 $ Propert_Owner : chr  "Y" "Y" "Y" "Y" ...
 $ CHILDREN      : int  0 2 0 2 2 0 0 0 0 0 ...
 $ Annual_income : num  180000 135000 157500 216000 202500 ...
 $ Family_Members: int  2 4 2 4 4 1 2 1 2 2 ...
 $ age           : int  51 43 27 42 30 43 41 44 55 25 ...
 $ Employed      : chr  "n" "y" "y" "y" ...
 $ EDUCATION     : chr  "Higher education" "Secondary / secondary special" "Secondary / secondary special" "Higher education" ...
 $ Marital_status: chr  "Married" "Married" "Married" "Married" ...
 $ Type_Income   : chr  "Pensioner" "Working" "Working" "State servant" ...
 $ Housing_type  : chr  "House / apartment" "House / apartment" "House / apartment" "House / apartment" ...
 $ label         : Factor w/ 2 levels "0","1": 2 1 1 1 1 1 1 1 1 1 ...


PUNTO 3: Investigar sobre las métricas de Precisión y Sensibilidad.
Las métricas de precisión y sensibilidad son indicadores que se utilizan para evaluar el rendimiento de modelos de SVM y Random Forests en problemas de clasificación.
->La precisión es una métrica que evalúa la proporción de predicciones correctas realizadas por el modelo entre todas las predicciones hechas para una clase específica.
En SVM, la precisión puede ser calculada después de la predicción del conjunto de datos de prueba. Para Random Forests puede calcular después de entrenar el modelo y evaluarlo en el conjunto de datos de prueba.
->La sensibilidad mide la proporción de instancias positivas que fueron correctamente identificadas por el modelo entre todas las instancias que son realmente positivas.
La sensibilidad en SVM evalúa qué tan bien el modelo identifica correctamente las muestras positivas.
En Random Forests se calcula después de la construcción del modelo y su evaluación. Al ser un conjunto de árboles de decisión, la sensibilidad mide la capacidad del modelo para capturar correctamente las instancias positivas.

PUNTO 4: Elaborar conclusiones en función de lo observado
Efecto del tamaño del conjunto de datos en SVM:
Se observa una tendencia en SVM donde un aumento en la cantidad de datos de entrenamiento conduce a un mejor rendimiento. Esto se refleja en una mejora en la precisión de las predicciones y clasificaciones del modelo. Esto sugiere que SVM se beneficia de conjuntos de datos más grandes para un entrenamiento más preciso.
Desempeño de SVM con dataset desbalanceado:
En los escenarios donde se trabajó con el dataset desbalanceado, se evidencia que SVM no logra una clasificación precisa. Se observa una inclinación hacia la clasificación de una sola clase, lo que indica que SVM puede tener dificultades al tratar con desbalanceo de clases, mostrando un rendimiento deficiente en la clasificación de clases minoritarias.
Comportamiento de Random Forest en dataset desbalanceado:
Contrario a lo observado en SVM, Random Forest muestra una capacidad superior para manejar datasets desbalanceados. Se destaca su habilidad para realizar predicciones más precisas incluso en escenarios donde las clases están desequilibradas. Esta característica demuestra la robustez de Random Forest ante desbalances de clases, lo que lo convierte en una opción más efectiva en tales situaciones en comparación con SVM.
